# Background Subtraction as an Optimization Problem

## Running This Notebook

### Setup
* Clone the [git repo](https://github.com/dmh43/background-subtraction) to a local directory.
* Update the `project_path` variable name below to the root of the project.
* Download video data from [here](http://www.ee.oulu.fi/~xliu/research/lsd/LSD.html).
* Install SPAMS from: [here](https://github.com/conda-forge/python-spams-feedstock) (if you use anaconda) or [here](http://spams-devel.gforge.inria.fr/downloads.html).
* Run `pip install -r requirements.txt` in the root directory of the project if you do not have all the dependecies.

### Getting Results
* Setting `video_name` with the name of the video to read will allow you to process any set of video frames from the above download link.
* Otherwise, the process will be run on the `WaterSurface` example
* The results will be written as a collection of images into the `background` and `foreground` folders, respectively.
* A gif of the background and foreground will also be created.

## Approach
This notebook investigates the formulation of background subtraction as a Robust PCA problem. Following the framework described in [1, 2], we implement two passes on the input video:
1. Identify candidate groups of pixels that belong to the foreground in each frame.
2. Remove pixels of low motion salience from the result of the first pass.

For the first pass, we reproduce the results from [1, 2, and 3] which formulate the problem as follows:
1. Solves the typical RPCA problem, using the L-1 norm.
2. Solves a variant of the RPCA problem using a structured-sparsity inducing norm.
3. Solves the non-convex formulation of the RPCA problem (does not use the nuclear norm as a convex surrogate for the rank).

In [1]:
from IPython.display import HTML, display
def display_anim(name):
    IMG_TAG = """<img src="data:image/gif;base64,{0}">"""
    data = open(name, "rb").read()
    data = data.encode("base64")
    return IMG_TAG.format(data)

In [ ]:
HTML(display_anim('structured_sparsity.gif'))

In [ ]:
HTML(display_anim('f.gif'))

In [ ]:
HTML(display_anim('l1.gif'))

In [2]:
from __future__ import print_function
import sys
import os

project_path = './'
video_name = ''
sys.path.append(project_path + 'src/')

In [3]:
import PIL.Image
import scipy
import scipy.sparse as sp
import scipy.io
import numpy as np
import numpy.linalg as la
import numpy.random as rn
import matplotlib.pyplot as plt

import frames as f
import graph as g
import motion as m
import group
from alm_lsd import inexact_alm_lsd, inexact_alm_bs

from FRPCA_GD import FRPCA
import cPickle as pickle
import os.path

In [4]:
# In order to read alternative data sources
def read_images(data_name):
    import glob
    import re
    
    def numericalSort(value):
        _numbers = re.compile(r'(\d+)')
        parts = _numbers.split(value)
        parts[1::2] = map(int, parts[1::2])
        return parts

    frame_path = project_path + 'data/' + data_name + '/'
    frames = []
    file_list = glob.glob(frame_path + "*.jpg")
    for file_names in sorted(file_list, key=numericalSort):
        image = PIL.Image.open(file_names)
        frame = np.asarray(image)
        frames.append(frame)
        
    return [0, len(frames)], np.array(frames)

In [5]:
def first_pass(frames_D, original_mean, frame_dimensions, downsampled_frame_dimensions, num_frames, norm_choice):
    if norm_choice == 'l1':
        print ('L1')
        background_L, foreground_S, err = inexact_alm_lsd(frames_D, None, 'l1')
        dims = downsampled_frame_dimensions
    elif norm_choice == 'structured_sparsity':
        print ('Structured Sparsity')
        batch_dimensions = [3, 3]
        graph = g.build_graph(downsampled_frame_dimensions, batch_dimensions)
        background_L, foreground_S, err = inexact_alm_lsd(frames_D, graph, 'structured_sparsity')
        dims = downsampled_frame_dimensions
    elif norm_choice == 'frpca':
        print ('FRPCA')
        background_L, foreground_S, err = FRPCA(frames_D, alpha = .3, r=1)
        dims = frame_dimensions
    else:
        raise ValueError('norm_choice is not valid.')

    # Masking first-RPCA foreground & Upsampling
    masked_S = f.foreground_mask(np.abs(foreground_S), frames_D, background_L)
    fg_frames = f.matrix_to_frames(masked_S, num_frames, dims)
    upsampled_fg = f.resize_frames(fg_frames, frame_dimensions)
    upsampled_fg = np.int32(upsampled_fg > 128) * 255

    # print
    bg_bin = f.restore_background(f.matrix_to_frames(background_L, num_frames,
                                                     dims), original_mean)
    bg_images = [PIL.Image.fromarray(frame) for frame in bg_bin]
    fg_images = [PIL.Image.fromarray(frame) for frame in upsampled_fg]
    for i in range(len(fg_images)):
        fg_images[i].save(project_path + "src/foreground_first_pass/out" + str(i) + norm_choice + ".gif")
        bg_images[i].save(project_path + "src/background_first_pass/out" + str(i) + norm_choice + ".gif")
    return upsampled_fg

In [6]:
frame_index = [0, 48]
video_data_path = project_path + 'LSD/data/WaterSurface.mat'
all_frames = scipy.io.loadmat(video_data_path)['ImData']
start_frame_index = frame_index[0]
end_frame_index = frame_index[1]
frames_to_process = np.rollaxis(all_frames[:, :, start_frame_index:end_frame_index], 2)

if video_name is not '':
    frame_index, frames_to_process = read_images(video_name)
frame_dimensions = frames_to_process.shape[1:]

downsampling_ratio = 1.0 / 1.0
downsampled_frames = f.resize_frames(frames_to_process, downsampling_ratio)
downsampled_frame_dimensions = downsampled_frames.shape[1:]

num_frames = downsampled_frames.shape[0]
normalized_frames, original_mean = f.normalize_and_center_frames(downsampled_frames)

frames_D = f.frames_to_matrix(normalized_frames, num_frames, downsampled_frame_dimensions)

print ('---Identifying Trajectories---')
if os.path.isfile('save.p'):
    trajectories = pickle.load(open( "save.p", "rb" ))
else:
    # find trajectory
    optical_flows = m.calc_forward_backward_flow(frames_to_process)
    trajectories = m.calc_trajectories(optical_flows[0], optical_flows[1], frame_dimensions, 5)
    pickle.dump(trajectories, open( "save.p", "wb" ))

for norm_choice in ['l1', 'structured_sparsity', 'frpca']:
    upsampled_fg = first_pass(frames_D, original_mean, frame_dimensions, downsampled_frame_dimensions, num_frames, norm_choice)

    print ('---Calculating Group Regularization Parameters---')
    # identify ground and compute lambda
    video_data_dimensions = [num_frames] + list(frame_dimensions)
    groups_info = group.find_groups(upsampled_fg, num_frames, upsampled_fg.shape[1:], min_size=50)
    m.set_groups_saliencies(groups_info, trajectories, video_data_dimensions, 5)
    m.set_regularization_lambdas(groups_info, video_data_dimensions)

    print ('---Group Sparse RPCA---')
    # group sparse RPCA
    normalized_frames, original_mean = f.normalize_and_center_frames(frames_to_process)
    frames_D = f.frames_to_matrix(normalized_frames, num_frames, frame_dimensions)
    final_L, final_S, err = inexact_alm_bs(frames_D, groups_info)

    # finalize S, L
    final_bg = f.restore_background(f.matrix_to_frames(final_L, num_frames, frame_dimensions), original_mean)
    masked_fg = f.foreground_mask(np.abs(final_S), frames_D, final_L)
    final_fg = f.matrix_to_frames(masked_fg, num_frames, frame_dimensions)

    # print
    fg_images = [PIL.Image.fromarray(frame) for frame in final_fg]
    bg_images = [PIL.Image.fromarray(frame) for frame in final_bg]

    for i in range(len(fg_images)):
        fg_images[i].save(project_path + "src/foreground/out" + str(i).zfill(3) + norm_choice + ".gif")
        bg_images[i].save(project_path + "src/background/out" + str(i).zfill(3) + norm_choice + ".gif")

    bg_images[0].save(project_path + norm_choice + "bg_out.gif", save_all=True, append_images=bg_images[1:])
    fg_images[0].save(project_path + norm_choice + "fg_out.gif", save_all=True, append_images=fg_images[1:])

---Identifying Trajectories---
L1
0 0.0205681310411
1 0.00651320377881
2 0.00565680103961
3 0.0050153551601
4 0.0046471504124
5 0.00442330272191
6 0.00431207407406
7 0.0042809186142
8 0.00430423380137
9 0.00436345495953
10 0.00443638439163
11 0.00450031726692
12 0.0045533549463
13 0.00459249873299
14 0.00462769297164
15 0.00464366539063
16 0.00466000679776
17 0.0046738777561
18 0.00467748876751
19 0.00471137331383
20 0.0047786565849
21 0.00495347690286
22 0.00519967783603
23 0.00544635994889
24 0.00559324796225
25 0.00554628797575
26 0.0053708244922
27 0.00522666579102
28 0.00511507430262
29 0.0049889383426
6.778943
---Calculating Group Regularization Parameters---
---Group Sparse RPCA---
Structured Sparsity
0 0.415564973574
1 0.387113607651
2 0.18078353615
3 0.14703866712
4 0.199616677182
5 0.196955603286
6 0.173207494761
7 0.169979782475
8 0.179573664115
9 0.146515949854
10 0.11792258778
11 0.0817834848313
12 0.0519751304719
13 0.0223661521349
14 0.0254970178901
15 0.0227895123649
16

In [34]:
HTML(display_anim('src/foreground_first_pass/out39structured_sparsity.gif'))

In [35]:
HTML(display_anim('src/foreground_first_pass/out39frpca.gif'))

In [36]:
HTML(display_anim('src/foreground_first_pass/out39l1.gif'))

## After First Pass
After the first pass, we see many artifacts scattered throughout the foreground image. This is expected due to the slight motion in the background that cannot be absorbed into the background image since we assume the background is low rank. Removing the low rank assumption gives degraded performance, including some "ghosts" in the background image. 

In [31]:
HTML(display_anim('src/foreground/out39structured_sparsity.gif'))

In [32]:
HTML(display_anim('src/foreground/out39frpca.gif'))

In [33]:
HTML(display_anim('src/foreground/out39l1.gif'))

## Final Results
After the second pass, we see that most of the artifacts have been removed, and in all three cases we have a relatively smooth border on the foreground image.